## Задание

**Вам необходимо решить задачу любыми способами, которые вы считаете верными.**

Задача: имеется предметная область, которая имеет иерархическую структуру показателей. Например, Город -> Улица -> Дом -> Подъезд -> Этаж -> Квартира -> Показание счетчика. Вам необходимо сгенерировать предметную область самостоятельно (которая вам нравится), представить ее в выбранной вами структуре данных. Написать функцию, которая ищет все доступные комбинации показателей. Функция должна быть реализована в переиспользуемом виде на случай, если предметная область изменится.

Ниже приложите все свои функции/классы, которыми вы пользовались для решения задачи. Также можете отправить дополнительно файлы, которые вы считаете нужны добавить. Продемонстрируйте работу ваших функций/классов.

---

ТЗ: 

1. Город
2. Город - Улица
3. Город - Улица - Дом
Словарь, поле city = 'moscow', street = 'uralskay',

Как нулевой элемент списка - входной словарь

Возвращаем словарь с восстановленным путем

И словари со всеми показателями следующего уровня

Слорваь вида {city: street: 'название', 

В каждом словаре

Список словарей (не список названий!!)

Словари все должны быть полными

Там где фильтрации нет - Nan.

Следующий этап - восстановление улицы.

---

In [140]:
from utils.get_dynamic_query_list import get_dynamic_query_list
from utils.get_query_list import get_query_list
from utils.get_rec_dynamic_query_list import get_rec_dynamic_query_list

In [128]:
# пример структуры
hierarchy = {
    'moscow': {
        'None':
            {'1':
                {
                'None': {
                        'вода': {}, 'электричество': {}
                    }  
                }, 
            '2':
                {
                'None': {
                        'вода': {}, 'электричество': {}
                        },
                '2': {
                        'вода': {}, 'электричество': {}
                        },
                '36': {
                        'вода': {}, 'электричество': {}
                        },
                '76': {
                        'вода': {}, 'электричество': {}
                        },
                },
            'None': 
                {
                '69': {
                        'вода': {}, 'электричество': {}
                    }
                }
        },
        'arbat':
            {'None':
                {
                    'None': 
                        {
                        'вода': {}, 'электричество': {}
                        }    
                }, 
            '2':
                {
                    'None': 
                    {
                        'вода': {}, 'электричество': {}
                    } 
                },
                
            '3':
                {
                    '4': 
                    {
                        'вода': {}, 'электричество': {}
                    }     
                },
            '7':
                {
                    'None': 
                    {
                        'вода': {}, 'электричество': {}
                    }
                }
        },
        'novoslobodskaya':
        {'2':
            {
               '3': {
                        'вода': {}, 'электричество': {}
                    } 
            },      
        '3':
            {
                'None': {
                    'вода': {}, 'электричество': {}
                }     
            }
        }
    },
    'spb': {
        'uralskaya': 
            {'2':
                {
                'None': {
                            'вода': {}, 'электричество': {}
                        } 
                },
                
            '3':
                {
                    'None': {
                        'вода': {}, 'электричество': {}
                    }     
                }
        },
        'rubinshteina':
        {'29':
            {
                'None': {
                        'вода': {}, 'электричество': {}
                    } 
            },
            
        '13':
            {
                'None': {
                    'вода': {}, 'электричество': {}
                }     
            }
        }
    }
}

In [133]:
import json

with open('data/hierarchy.json', 'w') as file:
    json.dump(hierarchy, fp=file, indent=5)


In [134]:
with open('data/hierarchy.json', 'r') as file:
    h = json.load(file)

In [135]:
h

{'moscow': {'None': {'1': {'None': {'вода': {}, 'электричество': {}}},
   '2': {'None': {'вода': {}, 'электричество': {}},
    '2': {'вода': {}, 'электричество': {}},
    '36': {'вода': {}, 'электричество': {}},
    '76': {'вода': {}, 'электричество': {}}},
   'None': {'69': {'вода': {}, 'электричество': {}}}},
  'arbat': {'None': {'None': {'вода': {}, 'электричество': {}}},
   '2': {'None': {'вода': {}, 'электричество': {}}},
   '3': {'4': {'вода': {}, 'электричество': {}}},
   '7': {'None': {'вода': {}, 'электричество': {}}}},
  'novoslobodskaya': {'2': {'3': {'вода': {}, 'электричество': {}}},
   '3': {'None': {'вода': {}, 'электричество': {}}}}},
 'spb': {'uralskaya': {'2': {'None': {'вода': {}, 'электричество': {}}},
   '3': {'None': {'вода': {}, 'электричество': {}}}},
  'rubinshteina': {'29': {'None': {'вода': {}, 'электричество': {}}},
   '13': {'None': {'вода': {}, 'электричество': {}}}}}}

In [70]:
query_1 = {'lvl_0': 'moscow', 'lvl_1': 'None', 'lvl_2': '2', 'lvl_3': 'None', 'lvl_4': 'вода'}
query_2 = {'lvl_0': 'moscow', 'lvl_1': 'None', 'lvl_2': 'None', 'lvl_3': 'None', 'lvl_4': 'вода'}
query_3 = {'lvl_0': 'moscow', 'lvl_1': 'arbat', 'lvl_2': '3', 'lvl_3': '4', 'lvl_4': 'вода'}
query_4 = {'lvl_0': 'moscow', 'lvl_1': 'None', 'lvl_2': '2', 'lvl_3': '36', 'lvl_4': 'вода'}
query_5 = {'lvl_0': 'moscow', 'lvl_1': 'None', 'lvl_2': 'None', 'lvl_3': '69', 'lvl_4': 'вода'}

In [122]:
get_query_list(hierarchy, query_1)

[{'lvl_0': 'moscow',
  'lvl_1': 'None',
  'lvl_2': '2',
  'lvl_3': 'None',
  'lvl_4': 'вода'},
 {'lvl_0': 'moscow',
  'lvl_1': 'arbat',
  'lvl_2': '2',
  'lvl_3': 'None',
  'lvl_4': 'вода'},
 {'lvl_0': 'moscow',
  'lvl_1': 'novoslobodskaya',
  'lvl_2': '2',
  'lvl_3': 'None',
  'lvl_4': 'вода'}]

---

### 2 Этап: Динамический обход

1 Версия: (хуже)

In [146]:
get_rec_dynamic_query_list(hierarchy, query_3)

{}

2 Версия:

In [123]:
get_dynamic_query_list(hierarchy, query_4)

[{'lvl_0': 'moscow',
  'lvl_1': 'None',
  'lvl_2': '2',
  'lvl_3': '36',
  'lvl_4': 'вода'}]

In [56]:
hierarchy

{'moscow': {'None': {'1': {'None': {'вода': {}, 'электричество': {}}},
   '2': {'None': {'вода': {}, 'электричество': {}},
    '2': {'вода': {}, 'электричество': {}},
    '36': {'вода': {}, 'электричество': {}},
    '76': {'вода': {}, 'электричество': {}}},
   'None': {'69': {'вода': {}, 'электричество': {}}}},
  'arbat': {'None': {'None': {'вода': {}, 'электричество': {}}},
   '2': {'None': {'вода': {}, 'электричество': {}}},
   '3': {'4': {'вода': {}, 'электричество': {}}},
   '7': {'None': {'вода': {}, 'электричество': {}}}},
  'novoslobodskaya': {'2': {'3': {'вода': {}, 'электричество': {}}},
   '3': {'None': {'вода': {}, 'электричество': {}}}}},
 'spb': {'uralskaya': {'2': {'None': {'вода': {}, 'электричество': {}}},
   '3': {'None': {'вода': {}, 'электричество': {}}}},
  'rubinshteina': {'2': {'None': {'вода': {}, 'электричество': {}}},
   '3': {'None': {'вода': {}, 'электричество': {}}}}}}

----

*Эталонный ответ:*

In [ ]:
answer = [{'lvl_0': 'moscow', 'lvl_1': 'None', 'lvl_2': 'None', 'lvl_3': 'None', 'driver_1': 'вода'}, 
          {'lvl_0': 'moscow', 'lvl_1': 'Arbat', 'lvl_2': 'None', 'lvl_3': 'None', 'driver_1': 'вода'},
          {'lvl_0': 'moscow', 'lvl_1': 'Kyzne4niy', 'lvl_2': 'None', 'lvl_3': 'None', 'driver_1': 'вода'},
          {'lvl_0': 'moscow', 'lvl_1': 'Lenina', 'lvl_2': 'None', 'lvl_3': 'None', 'driver_1': 'вода'},
          {'lvl_0': 'moscow', 'lvl_1': 'Pushkina', 'lvl_2': 'None', 'lvl_3': 'None', 'driver_1': 'вода'},
          {'lvl_0': 'moscow', 'lvl_1': 'Arbat', 'lvl_2': '1', 'lvl_3': 'None', 'driver_1': 'вода'},
          {'lvl_0': 'moscow', 'lvl_1': 'Arbat', 'lvl_2': '2', 'lvl_3': 'None', 'driver_1': 'вода'},
          {'lvl_0': 'moscow', 'lvl_1': 'Arbat', 'lvl_2': '3', 'lvl_3': 'None', 'driver_1': 'вода'},
          ...
          ]